# Add ELAs from AARs to snowline files

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import ee
import sys

In [ ]:
# -----Define paths in directory
# path to study-sites
study_sites_path = '/Users/raineyaberle/Google Drive/My Drive/Research/CryoGARS-Glaciology/Advising/student-research/Alexandra-Friel/snow_cover_mapping_application/study-sites/'
# path to snow-cover-mapping
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'
# add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

In [ ]:
# -----Initialize GEE account
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except: 
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# -----Grab list of site names in study_sites_path
site_names = sorted(os.listdir(study_sites_path))
site_names = [x for x in site_names if not x.startswith('.')]
# don't include BGOTUS (already done)
bgotus_site_names = ['Wolverine', 'LemonCreek', 'Gulkana', 'SouthCascade', 'Sperry']
site_names = [x for x in site_names if (x not in bgotus_site_names) and ("all_" not in x)]
print(str(len(site_names)) + ' study sites:')
site_names

In [ ]:
# iterate over site names
for site_name in tqdm(site_names):
    
    print(site_name)
    
    # Load snowline file names
    snowlines_path = study_sites_path + site_name + '/imagery/snowlines/'
    snowline_fns = [os.path.basename(x) for x in glob.glob(snowlines_path + '*.csv')]
    if len(snowline_fns)<1:
        print(' ')
        continue
    
    # Load AOI
    AOI_path = study_sites_path + site_name + '/AOIs/'
    AOI_fn = glob.glob(AOI_path + '*_outline.shp')[0]
    AOI = gpd.read_file(AOI_fn)
    AOI_WGS = AOI.to_crs('EPSG:4326')
    AOI_WGS_centroid = [AOI_WGS.geometry[0].centroid.xy[0][0],
                        AOI_WGS.geometry[0].centroid.xy[1][0]]
    # grab the optimal UTM zone EPSG code
    epsg_UTM = f.convert_wgs_to_utm(AOI_WGS_centroid[0], AOI_WGS_centroid[1])
    # reproject AOI to the optimal UTM zone
    AOI_UTM = AOI.to_crs('EPSG:'+epsg_UTM)

    # Load DEM, clip to AOI
    DEM_path = study_sites_path + site_name + '/DEMs/'
    # DEM_fn = glob.glob(DEM_path + '*USGS*.tif')[0]
    # DEM = rxr.open_rasterio(DEM_fn)
    DEM = f.query_gee_for_dem(AOI_UTM, base_path, site_name, DEM_path)  
    if len(np.shape(DEM.elevation.data))>2:
        DEM['elevation'] = DEM.elevation[0]
    DEM_clip = DEM.rio.clip(AOI_UTM.geometry, 'EPSG:' + str(epsg_UTM))
    # DEM_clip = xr.where(DEM_clip < 250, np.nan, DEM_clip)
    elevations = np.ravel(DEM_clip.elevation.data)    
    plt.hist(elevations)
    plt.show()
    
    # iterate over snowline files
    for snowline_fn in tqdm(snowline_fns):
                        
        # load snowline
        try:
            snowline = pd.read_csv(snowlines_path + snowline_fn)
        except:
            continue
            
        cols = snowline.columns.tolist()
        # if 'ELA_from_AAR_m' in cols:
        #     # print('column already exists, skipping...')
        #     # print(' ')
        #     continue

        # calculate ELA from AAR
        aar = snowline['AAR'].values[0]
        ela_from_aar = np.nanquantile(elevations, 1 - aar)

        # add to snowline dataframe
        snowline['ELA_from_AAR_m'] = ela_from_aar

        # remove any "Unnamed" columns
        cols = snowline.columns.tolist()
        cols = [x for x in cols if "Unnamed" not in x]
        snowline = snowline[cols]
                
        # save to file
        snowline.to_csv(snowlines_path + snowline_fn, index=False)
        
    print(' ')
                
    

### Plot snowlines and ELAs from AARs

In [ ]:
plt.rcParams.update({'font.size':12, 'font.sans-serif':'Arial'})

# iterate over site names
for site_name in site_names:
        
    # define paths in directory
    snowlines_path = study_sites_path + site_name + '/imagery/snowlines/'
    
    # load snowline and classified image file names
    snowline_fns = [os.path.basename(x) for x in glob.glob(snowlines_path + '*.csv')]
    
    if len(snowline_fns) < 1:
        continue
    
    # iterate over snowline files
    snowlines = pd.DataFrame()
    for snowline_fn in snowline_fns:
                
        # load snowline
        try:
            snowline = pd.read_csv(snowlines_path + snowline_fn)
            snowlines = pd.concat([snowlines, snowline])
        except:
            continue
    
    snowlines.reset_index(drop=True, inplace=True)
    snowlines['datetime'] = pd.to_datetime(snowlines['datetime'], format='mixed')
    
    # plot
    fig, ax = plt.subplots(1,1,figsize=(10,6))
    ax.plot(snowlines['datetime'], snowlines['snowline_elevs_median_m'], '.m', label='from snowline')
    ax.plot(snowlines['datetime'], snowlines['ELA_from_AAR_m'], 'xb', markersize=5, label='from AAR')
    ax.grid()
    ax.set_title(site_name)
    ax.legend(loc='upper left')
    ax.set_ylabel('Elevation [m]')
    plt.show()
    
    # fig.savefig('ELAs_' + site_name + '.png')
    